In [1]:
import pandas as pd
import os

from datasets import YagoECQ
from utils import format_query, extract_name_from_yago_uri

In [2]:
uri = "reverse-http://schema.org/leader"
kb_name, relation = extract_name_from_yago_uri(uri)
cc = YagoECQ(subname=f"{kb_name}:{relation}", query_id=uri, entity_types=["entities", "fake_entities"], max_contexts=100)

Failed to load entities, contexts, and queries from paths None, None, and None.
Manually reconstructing dataset and saving to aforementioned paths.


In [3]:
qe_df = cc.get_contexts_per_query_entity_df()
print(qe_df.info())
print(qe_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   q_id        4000 non-null   object
 1   query_form  4000 non-null   object
 2   entity      4000 non-null   object
 3   answer      4000 non-null   object
 4   contexts    4000 non-null   object
dtypes: object(5)
memory usage: 156.4+ KB
None
                               q_id  \
0  reverse-http://schema.org/leader   
1  reverse-http://schema.org/leader   
2  reverse-http://schema.org/leader   
3  reverse-http://schema.org/leader   
4  reverse-http://schema.org/leader   

                                   query_form               entity  \
0  Q: Is {entity} the leader of {answer}?\nA:     (Étienne Blanc,)   
1  Q: Is {entity} the leader of {answer}?\nA:  (Laurent Wauquiez,)   
2  Q: Is {entity} the leader of {answer}?\nA:     (Bernard Piras,)   
3  Q: Is {entity} the leader of {answer}?\nA:   (Ime

In [4]:
qc_df = cc.get_entities_per_query_context_df()
print(qc_df.info())
print(qc_df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   q_id        200 non-null    object
 1   query_form  200 non-null    object
 2   context     200 non-null    object
 3   entities    200 non-null    object
dtypes: object(4)
memory usage: 6.4+ KB
None
                               q_id  \
0  reverse-http://schema.org/leader   
1  reverse-http://schema.org/leader   
2  reverse-http://schema.org/leader   
3  reverse-http://schema.org/leader   
4  reverse-http://schema.org/leader   

                                   query_form  \
0  Q: Is {entity} the leader of {answer}?\nA:   
1  Q: Is {entity} the leader of {answer}?\nA:   
2  Q: Is {entity} the leader of {answer}?\nA:   
3  Q: Is {entity} the leader of {answer}?\nA:   
4  Q: Is {entity} the leader of {answer}?\nA:   

                                             context  \
0  Olga Sukhenko is the l